In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import time
import numpy as np
import torch
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('.')) + '/src'
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from criteria import KIDMindLoss
from helpers import Trainer
from helpers import Logger
from helpers import Trial
from mibi_dataloader import KID_Data
from modules import Mind_of_KID
import utils

In [ ]:
# Load the data
main_dir = '/home/hazmat/GitHub/KID/'
train_dir = main_dir + 'datasets/KID_memory'
# test_dir = main_dir + 'low_AD/'
modl_dir = main_dir + 'models/'
rslt_dir = main_dir + 'results/'

train_ds = KID_Data(folder=train_dir, crop=55, scale=1, stride=1)
# test_ds = MIBIData(folder=test_dir, crop=32, scale=10, stride=16)

In [ ]:
laddernet_args = dict()

# kidmind = Mind_of_KID()
kidmind = Mind_of_KID.load_model(main_dir + 'models/', 'model_2999')
if torch.cuda.is_available():
    kidmind.cuda()
    pass

In [ ]:
kidmind.set_noise_level(0)
kidmind.eval()

idx1 = 20
idx2 = 48
batch1 = train_ds.get_samples([idx1])
batch2 = train_ds.get_samples([idx2])

batch1 = kidmind.forward(**batch1)
batch2 = kidmind.forward(**batch2)

img1 = batch1['~s_i'][0,:,:,:].transpose(0,1).transpose(1,2).detach()
img2 = batch2['~s_i'][0,:,:,:].transpose(0,1).transpose(1,2).detach()

p = batch2['p_i']
print(p)

fig = plt.figure(figsize=[10,5])
ax1 = plt.subplot(1,2,1)
ax2 = plt.subplot(1,2,2)

ax1.imshow(img1.cpu())
ax2.imshow(img2.cpu())

In [ ]:
genetic_args = dict()
genetic_args['pop_size'] = 100
genetic_args['seq_length'] = 3
genetic_args['generations'] = 25
genetic_args['steps'] = 1
genetic_args['learning_rate'] = 1
genetic_args['n_cut'] = 10
genetic_args['n_children'] = 9
genetic_args['noise_lvl'] = 0.2
genetic_args['optimize'] = True

clip_val = 0.5

In [ ]:
num_seqs = 10
import time
import kid_ga
t = time.time()
r_target, p_t, p_next = kid_ga.build_dataset(num_seqs, train_ds, kidmind, clip_val, genetic_args)
time_passed = time.time() - t
print(time_passed, ' seconds')
print(time_passed / 60 / 60 / 24, ' hours')

In [ ]:
raw_seq_data = dict()
raw_seq_data['r_target'] = r_target
raw_seq_data['p_t'] = p_t
raw_seq_data['p_next'] = p_next

import pickle
pickle.dump(raw_seq_data, open(main_dir + 'raw_seq_data.p', 'wb'))